## Data Glance and Cleaning

In [63]:
#import packages
import numpy as np
import pandas as pd

In [4]:
#data exam
user_datasample = pd.read_csv('user_log_format1.csv')
user_datasample.head()

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
0,328862,323294,833,2882,2661.0,829,0
1,328862,844400,1271,2882,2661.0,829,0
2,328862,575153,1271,2882,2661.0,829,0
3,328862,996875,1271,2882,2661.0,829,0
4,328862,1086186,1271,1253,1049.0,829,0


In [5]:
#find out the missing value in this file
user_datasample.isnull().sum()

user_id            0
item_id            0
cat_id             0
seller_id          0
brand_id       91015
time_stamp         0
action_type        0
dtype: int64

In [10]:
#get the mode brand id for all stores, fillin NA value in brain_id column
missing = user_datasample[user_datasample.brand_id.isnull()].index
seller = user_datasample.groupby(['seller_id']).apply(lambda x:x.brand_id.mode()[0]).reset_index()
get_brand = user_datasample.loc[missing]
get_brand = get_brand.merge(seller,how='left',on=['seller_id'])[0].astype('float32')
get_brand.index = missing
user_datasample.loc[missing,'brand_id'] = get_brand

In [20]:
#see whether null values are filled
user_datasample.isnull().sum()

user_id        0
item_id        0
cat_id         0
seller_id      0
brand_id       0
time_stamp     0
action_type    0
dtype: int64

In [7]:
#user info
user_info = pd.read_csv('user_info_format1.csv')
user_info.head()

,user_id,age_range,gender
0,376517,6.0,1.0
1,234512,5.0,0.0
2,344532,5.0,0.0
3,186135,5.0,0.0
4,30230,5.0,0.0


In [8]:
#user info
user_info.isnull().sum()

user_id         0
age_range    2217
gender       6436
dtype: int64

In [9]:
user_info.age_range.fillna(user_info.age_range.median(),inplace=True)
user_info.gender.fillna(2,inplace=True)
#user info NA value exam
user_info.isnull().sum()

user_id      0
age_range    0
gender       0
dtype: int64

In [16]:
# seller and item numbers
item_count = user_datasample[['seller_id','item_id']]
item_count = item_count.groupby(['seller_id'])['item_id'].nunique().reset_index()
item_count.head()
item_count.columns = ['seller_id', 'item_count']
item_count.head()

,seller_id,item_count
0,1,2977
1,2,154
2,3,171
3,4,155
4,5,660


In [18]:
# seller and categories count
cat_count = user_datasample[['seller_id','cat_id']]
cat_count = cat_count.groupby(['seller_id'])['cat_id'].nunique().reset_index()
cat_count.head()
cat_count.columns = ['seller_id', 'cat_count']
cat_count.head()

,seller_id,cat_count
0,1,44
1,2,10
2,3,4
3,4,7
4,5,23


In [19]:
# seller and brand numbers
brand_count = user_datasample[['seller_id','brand_id']]
brand_count = brand_count.groupby(['seller_id'])['brand_id'].nunique().reset_index()
brand_count.head()
brand_count.columns = ['seller_id', 'brand_count']
brand_count.head()

,seller_id,brand_count
0,1,2
1,2,1
2,3,1
3,4,2
4,5,1


In [38]:
repeat_purchase = user_datasample[(user_datasample['action_type'] == 2) & (user_datasample['time_stamp'] < 1111)]
repeat_purchase = repeat_purchase.groupby(['seller_id'])['user_id'].value_counts().to_frame()
repeat_purchase.head()

user_id
seller_id user_id         
1         406           21
          56832         21
          180072        20
          339584        15
          88181         14

In [39]:
repeat_purchase.columns = ['purchase_count']
repeat_purchase.reset_index(inplace=True)
repeat_purchase.head()

,seller_id,user_id,purchase_count
0,1,406,21
1,1,56832,21
2,1,180072,20
3,1,339584,15
4,1,88181,14


In [40]:
repeat_purchase = repeat_purchase[repeat_purchase['purchase_count'] > 1]
repeat_purchase = repeat_purchase.groupby(['seller_id'])['user_id'].nunique().reset_index()

In [43]:
repeat_purchase.describe()

,seller_id,user_id
count,4888.000000,4888.000000
mean,2492.389321,75.873159
std,1441.626649,151.651987
min,1.000000,1.000000
25%,1244.750000,11.000000
50%,2487.500000,29.000000
75%,3740.250000,75.000000
max,4995.000000,2748.000000


In [45]:
# Because some sellers without repeated purchases are excluded,we have to add them and fill 0 for future merge.
all_sellerid = pd.DataFrame({'seller_id':range(1, 4996 ,1)})
repeat_purchase = pd.merge(all_sellerid,repeat_purchase,how='left',on=['seller_id']).fillna(0)
repeat_purchase.head()

,seller_id,user_id
0,1,2214.0
1,2,4.0
2,3,2.0
3,4,48.0
4,5,14.0


In [97]:
repeat_purchase.columns = ['seller_id', 'repeat_users_count']
repeat_purchase.head()

,seller_id,repeat_users_count
0,1,2214.0
1,2,4.0
2,3,2.0
3,4,48.0
4,5,14.0


In [49]:
# seller_id and action_type
actions = user_datasample[['seller_id', 'action_type']]
actions.head()

,seller_id,action_type
0,2882,0
1,2882,0
2,2882,0
3,2882,0
4,1253,0


In [50]:
# extract each action type
dummy_action = pd.get_dummies(actions['action_type'], prefix='action')
dummy_action.head()

,action_0,action_1,action_2,action_3
0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0


In [56]:
# count the number of each actions for each seller
seller_actions = pd.concat([actions,dummy_action], axis=1).groupby(['seller_id'], as_index=False).sum()
#.drop('action_type', axis=1, inplace=True)

In [57]:
seller_actions.head()

,seller_id,action_type,action_0,action_1,action_2,action_3
0,1,74119,308236.0,444.0,17705.0,12755.0
1,2,818,2030.0,8.0,189.0,144.0
2,3,663,2399.0,4.0,67.0,175.0
3,4,1082,2646.0,2.0,294.0,164.0
4,5,1965,7483.0,9.0,144.0,556.0


In [60]:
seller_actions.drop('action_type',1, inplace=True)

In [61]:
seller_actions.head()

,seller_id,action_0,action_1,action_2,action_3
0,1,308236.0,444.0,17705.0,12755.0
1,2,2030.0,8.0,189.0,144.0
2,3,2399.0,4.0,67.0,175.0
3,4,2646.0,2.0,294.0,164.0
4,5,7483.0,9.0,144.0,556.0


In [66]:
seller_actions['click_to_buy_ratio'] = np.log1p(seller_actions['action_2']) - np.log1p(seller_actions['action_0'])
seller_actions['cart_to_buy_ratio'] = np.log1p(seller_actions['action_2']) - np.log1p(seller_actions['action_1'])
seller_actions['fav_to_buy_ratio'] = np.log1p(seller_actions['action_2']) - np.log1p(seller_actions['action_3'])

In [67]:
seller_actions.head()

,seller_id,action_0,action_1,action_2,action_3,click_to_buy_ratio,cart_to_buy_ratio,fav_to_buy_ratio
0,1,308236.0,444.0,17705.0,12755.0,-2.856965,3.683585,0.327902
1,2,2030.0,8.0,189.0,144.0,-2.369259,3.049799,0.270290
2,3,2399.0,4.0,67.0,175.0,-3.563716,2.610070,-0.950976
3,4,2646.0,2.0,294.0,164.0,-2.194207,4.588363,0.581030
4,5,7483.0,9.0,144.0,556.0,-3.943789,2.674149,-1.345831


In [79]:
# user_id, seller_id and action_type
user_actions = user_datasample[['seller_id', 'user_id', 'action_type']]
user_actions.head()

,seller_id,user_id,action_type
0,2882,328862,0
1,2882,328862,0
2,2882,328862,0
3,2882,328862,0
4,1253,328862,0


In [80]:
dummy_user = pd.get_dummies(user_actions['action_type'], prefix='seller_user_action')
dummy_user.head()

,seller_user_action_0,seller_user_action_1,seller_user_action_2,seller_user_action_3
0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0


In [81]:
seller_user_action = pd.concat([user_actions, dummy_user], axis=1)
seller_user_action.drop('action_type',1, inplace=True)

In [82]:
seller_user_action.head()

,seller_id,user_id,seller_user_action_0,seller_user_action_1,seller_user_action_2,seller_user_action_3
0,2882,328862,1.0,0.0,0.0,0.0
1,2882,328862,1.0,0.0,0.0,0.0
2,2882,328862,1.0,0.0,0.0,0.0
3,2882,328862,1.0,0.0,0.0,0.0
4,1253,328862,1.0,0.0,0.0,0.0


In [83]:
# drop duplicates so that we can count the number of users who made actions for each seller
seller_user_action.drop_duplicates(inplace=True)

In [84]:
seller_user_action.head()

,seller_id,user_id,seller_user_action_0,seller_user_action_1,seller_user_action_2,seller_user_action_3
0,2882,328862,1.0,0.0,0.0,0.0
4,1253,328862,1.0,0.0,0.0,0.0
7,883,328862,1.0,0.0,0.0,0.0
10,420,328862,1.0,0.0,0.0,0.0
11,4605,328862,1.0,0.0,0.0,0.0


In [89]:
dummy_user_1 = seller_user_action.groupby(['seller_id'])['seller_user_action_0'].sum()
dummy_user_2 = seller_user_action.groupby(['seller_id'])['seller_user_action_1'].sum()
dummy_user_3 = seller_user_action.groupby(['seller_id'])['seller_user_action_2'].sum()
dummy_user_4 = seller_user_action.groupby(['seller_id'])['seller_user_action_3'].sum()

In [90]:
seller_user_action = pd.concat([dummy_user_1, dummy_user_2, dummy_user_3, dummy_user_4], axis=1).reset_index()

In [91]:
seller_user_action.head()

,seller_id,seller_user_action_0,seller_user_action_1,seller_user_action_2,seller_user_action_3
0,1,29251.0,265.0,7666.0,4965.0
1,2,902.0,6.0,161.0,127.0
2,3,1103.0,4.0,65.0,150.0
3,4,1384.0,2.0,201.0,153.0
4,5,3535.0,9.0,120.0,458.0


In [92]:
seller_actions = pd.merge(seller_actions,seller_user_action,on=['seller_id'])

In [93]:
seller_actions = pd.merge(seller_actions,item_count,on=['seller_id'])

In [94]:
seller_actions = pd.merge(seller_actions,cat_count,on=['seller_id'])

In [95]:
seller_actions = pd.merge(seller_actions,brand_count,on=['seller_id'])

In [98]:
seller_actions = pd.merge(seller_actions,repeat_purchase,on=['seller_id'])

In [99]:
seller_actions.head()

,seller_id,action_0,action_1,action_2,action_3,click_to_buy_ratio,cart_to_buy_ratio,fav_to_buy_ratio,seller_user_action_0,seller_user_action_1,seller_user_action_2,seller_user_action_3,item_count,cat_count,brand_count,repeat_users_count
0,1,308236.0,444.0,17705.0,12755.0,-2.856965,3.683585,0.327902,29251.0,265.0,7666.0,4965.0,2977,44,2,2214.0
1,2,2030.0,8.0,189.0,144.0,-2.369259,3.049799,0.270290,902.0,6.0,161.0,127.0,154,10,1,4.0
2,3,2399.0,4.0,67.0,175.0,-3.563716,2.610070,-0.950976,1103.0,4.0,65.0,150.0,171,4,1,2.0
3,4,2646.0,2.0,294.0,164.0,-2.194207,4.588363,0.581030,1384.0,2.0,201.0,153.0,155,7,2,48.0
4,5,7483.0,9.0,144.0,556.0,-3.943789,2.674149,-1.345831,3535.0,9.0,120.0,458.0,660,23,1,14.0


In [106]:
# calculate the ratio of item, category, brand of each seller of total.
seller_actions.item_ratio = seller_actions.item_count/user_datasample.item_id.nunique()
seller_actions.cat_ratio = seller_actions.cat_count/user_datasample.cat_id.nunique()
seller_actions.brand_ratio = seller_actions.brand_count/user_datasample.brand_id.nunique()

In [107]:
seller_actions.head()

,seller_id,action_0,action_1,action_2,action_3,click_to_buy_ratio,cart_to_buy_ratio,fav_to_buy_ratio,seller_user_action_0,seller_user_action_1,seller_user_action_2,seller_user_action_3,item_count,cat_count,brand_count,repeat_users_count,item_ratio,cat_ratio,brand_ratio
0,1,308236.0,444.0,17705.0,12755.0,-2.856965,3.683585,0.327902,29251.0,265.0,7666.0,4965.0,2977,44,2,2214.0,0.002730,0.026538,0.000237
1,2,2030.0,8.0,189.0,144.0,-2.369259,3.049799,0.270290,902.0,6.0,161.0,127.0,154,10,1,4.0,0.000141,0.006031,0.000118
2,3,2399.0,4.0,67.0,175.0,-3.563716,2.610070,-0.950976,1103.0,4.0,65.0,150.0,171,4,1,2.0,0.000157,0.002413,0.000118
3,4,2646.0,2.0,294.0,164.0,-2.194207,4.588363,0.581030,1384.0,2.0,201.0,153.0,155,7,2,48.0,0.000142,0.004222,0.000237
4,5,7483.0,9.0,144.0,556.0,-3.943789,2.674149,-1.345831,3535.0,9.0,120.0,458.0,660,23,1,14.0,0.000605,0.013872,0.000118


In [108]:
# calculate the ratio of clicks, add-to-cart, purchase, add-to-fav of each seller of total activities.
seller_actions.click_people_ratio = seller_actions.seller_user_action_0/user_datasample[user_datasample.action_type == 0]['user_id'].nunique()
seller_actions.add_people_ratio = seller_actions.seller_user_action_1/user_datasample[user_datasample.action_type == 1]['user_id'].nunique()
seller_actions.add_people_ratio = seller_actions.seller_user_action_2/user_datasample[user_datasample.action_type == 2]['user_id'].nunique()
seller_actions.add_people_ratio = seller_actions.seller_user_action_3/user_datasample[user_datasample.action_type == 3]['user_id'].nunique()

In [109]:
seller_actions.head()

,seller_id,action_0,action_1,action_2,action_3,click_to_buy_ratio,cart_to_buy_ratio,fav_to_buy_ratio,seller_user_action_0,seller_user_action_1,seller_user_action_2,seller_user_action_3,item_count,cat_count,brand_count,repeat_users_count,item_ratio,cat_ratio,brand_ratio
0,1,308236.0,444.0,17705.0,12755.0,-2.856965,3.683585,0.327902,29251.0,265.0,7666.0,4965.0,2977,44,2,2214.0,0.002730,0.026538,0.000237
1,2,2030.0,8.0,189.0,144.0,-2.369259,3.049799,0.270290,902.0,6.0,161.0,127.0,154,10,1,4.0,0.000141,0.006031,0.000118
2,3,2399.0,4.0,67.0,175.0,-3.563716,2.610070,-0.950976,1103.0,4.0,65.0,150.0,171,4,1,2.0,0.000157,0.002413,0.000118
3,4,2646.0,2.0,294.0,164.0,-2.194207,4.588363,0.581030,1384.0,2.0,201.0,153.0,155,7,2,48.0,0.000142,0.004222,0.000237
4,5,7483.0,9.0,144.0,556.0,-3.943789,2.674149,-1.345831,3535.0,9.0,120.0,458.0,660,23,1,14.0,0.000605,0.013872,0.000118
